## Load the data

In [1]:
using DataDrop

In [2]:
P_max = DataDrop.retrieve_matrix("PTDF_data/P_max_gen.h5");

In [3]:
P_total = DataDrop.retrieve_matrix("PTDF_data/P_total.h5");

In [4]:
A_constraints = DataDrop.retrieve_matrix("PTDF_data/A_gen_total.h5");

In [5]:
P_constraints = DataDrop.retrieve_matrix("PTDF_data/gen_total.h5");

In [6]:
A_ramp = DataDrop.retrieve_matrix("PTDF_data/A_gen_ramp.h5");

In [7]:
ΔP_ramp = DataDrop.retrieve_matrix("PTDF_data/gen_ramp.h5");

In [8]:
L_lines = DataDrop.retrieve_matrix("PTDF_data/linear_line_cost.h5");

In [9]:
L_gens = DataDrop.retrieve_matrix("PTDF_data/linear_gen_cost.h5");

In [10]:
Q = DataDrop.retrieve_matrix("PTDF_data/quadratic_cost.h5");

## Computation

In [11]:
using MiniLoggers

logger = MiniLogger(format="[{timestamp:blue}] {group:red:bold} {message}")
global_logger(logger);

In [12]:
import MathOptInterface as MOI

In [13]:
using Gurobi

const gurobi_env = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-17


In [14]:
# optimizer = MOI.instantiate(MOI.OptimizerWithAttributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))

In [15]:
optimizer = Gurobi.Optimizer(gurobi_env)

    sense  : minimize
    number of variables             = 0
    number of linear constraints    = 0
    number of quadratic constraints = 0
    number of sos constraints       = 0
    number of non-zero coeffs       = 0
    number of non-zero qp objective terms  = 0
    number of non-zero qp constraint terms = 0


In [16]:
function opf(Q::AbstractArray{<:Real,2}, L::AbstractArray{<:Real,2}, 
        P_max::AbstractVector{<:Real}, P_total::AbstractVector{<:Real},
        A_constraints::AbstractArray{<:Real,2}, P_constraints::AbstractVector{<:Real},
        A_ramp::AbstractArray{<:Real,2} = Array{Real}(undef, 0, 0), ΔP_ramp::AbstractVector{<:Real} = Real[],
        P_ramp_first::AbstractVector{<:Real} = Real[], P_ramp_last::AbstractVector{<:Real} = Real[];
        log_group::String = "")
    
    N = length(P_max)
    T = length(P_total)
    n_constraints = length(P_constraints)
    n_ramp = length(ΔP_ramp)

    # check dimensions of the input
    @assert size(Q) == (N, N)
    @assert size(L) == (N, T)
    @assert size(A_constraints) == (n_constraints, N)
    @assert (size(A_ramp) == (n_ramp, N)) || (n_ramp == 0)
    @assert length(P_ramp_first) ∈ [0, n_ramp]
    @assert length(P_ramp_last) == length(P_ramp_first)

    ramp_constraint_type = length(P_ramp_first) == 0 ? "cyclic" : "fixed boundaries"
    @info ("OPF with $T time steps, $N generators, $n_constraints annual constraints, " *
        "and $n_ramp ramp constraints ($ramp_constraint_type)") _group = log_group
    log_group = " "^length(log_group)
    
    # check feasibility of the model
    @info " -> checking model" _group = log_group
    @assert all(P_constraints .<= A_constraints * P_max)
    @assert all(ΔP_ramp .>= 0)
    @assert all(P_total .<= sum(P_max))

    # variables
    @info " -> defining variables" _group = log_group
    P_vec = MOI.add_variables(optimizer, N * T)
    P = reshape(P_vec, (N, T))

    # constraints 
    @info " -> defining constraints" _group = log_group
    MOI.add_constraints(optimizer, P_vec, [MOI.Interval(0.0, P_max[i]) for t = 1:T for i = 1:N])

    MOI.add_constraints(optimizer,
        [MOI.ScalarAffineFunction([MOI.ScalarAffineTerm(1.0, P[i,t]) for i = 1:N], 0.0) for t = 1:T],
        [MOI.EqualTo(P_total[t]) for t = 1:T])
    MOI.add_constraints(optimizer,
        A_constraints * [MOI.ScalarAffineFunction([MOI.ScalarAffineTerm(1.0 / T, P[i,t]) for t = 1:T], 0.0) for i = 1:N],
        [MOI.EqualTo(P_constraints[n]) for n = 1:n_constraints])
    
    if n_ramp > 0
        P_ramp = A_ramp * P
        if length(P_ramp_first) == 0
            ΔP = [P_ramp[n, t] - P_ramp[n, t % T + 1] for t = 1:T for n = 1:n_ramp]
            MOI.add_constraints(optimizer, ΔP, [MOI.GreaterThan(-ΔP_ramp[n]) for t = 1:T for n = 1:n_ramp])
            MOI.add_constraints(optimizer, ΔP, [MOI.LessThan(ΔP_ramp[n]) for t = 1:T for n = 1:n_ramp])
        else
            ΔP = [P_ramp[n, t] - P_ramp[n, t + 1] for t = 1:T-1 for n = 1:n_ramp]
            MOI.add_constraints(optimizer, ΔP, [MOI.GreaterThan(-ΔP_ramp[n]) for t = 1:T-1 for n = 1:n_ramp])
            MOI.add_constraints(optimizer, ΔP, [MOI.LessThan(ΔP_ramp[n]) for t = 1:T-1 for n = 1:n_ramp])
            P_first = [P_ramp[n, 1] for n = 1:n_ramp]
            MOI.add_constraints(optimizer, P_first, [MOI.GreaterThan(P_ramp_first[n] - ΔP_ramp[n]) for n = 1:n_ramp])
            MOI.add_constraints(optimizer, P_first, [MOI.LessThan(P_ramp_first[n] + ΔP_ramp[n]) for n = 1:n_ramp])
            P_last = [P_ramp[n, T] for n = 1:n_ramp]
            MOI.add_constraints(optimizer, P_last, [MOI.GreaterThan(P_ramp_last[n] - ΔP_ramp[n]) for n = 1:n_ramp])
            MOI.add_constraints(optimizer, P_last, [MOI.LessThan(P_ramp_last[n] + ΔP_ramp[n]) for n = 1:n_ramp])
        end
    end
    
    @info " -> computing objective function" _group = log_group
    quadratic_terms = vcat(
        [MOI.ScalarQuadraticTerm(2.0 * Q[i,i], P[i, t], P[i, t]) for i = 1:N for t = 1:T],
        [MOI.ScalarQuadraticTerm(Q[i,j], P[i, t], P[j, t]) for i = 1:N for j = (i+1):N for t = 1:T]
    )
    affine_terms = [MOI.ScalarAffineTerm(L[i, t], P[i, t]) for i = 1:N for t = 1:T]
    objective = MOI.ScalarQuadraticFunction(quadratic_terms, affine_terms, 0.0)
    
    MOI.set(optimizer, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), objective)
    MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    
    @info " -> optimizing" _group = log_group
    MOI.optimize!(optimizer)
    
    
    @info " -> exporting results" _group = log_group
    P_vec_solution = MOI.get(optimizer, MOI.VariablePrimal(), P_vec)
    P_vec_solution = map(x -> isapprox(x, 0, atol=1e-6) ? 0.0 : x, P_vec_solution)
    P_solution = reshape(P_vec_solution, (N, T))
    
    return P_solution
end

opf (generic function with 5 methods)

In [17]:
function partitioned_opf(partitions::Vector{Int},
        Q::AbstractArray{<:Real,2}, L_lines::AbstractArray{<:Real,2}, L_gens::AbstractArray{<:Real,2}, 
        P_max::AbstractVector{<:Real}, P_total::AbstractVector{<:Real},
        A_constraints::AbstractArray{<:Real,2}, P_constraints::AbstractVector{<:Real},
        A_ramp::AbstractArray{<:Real,2} = Array{Real}(undef, 0, 0), ΔP_ramp::AbstractVector{<:Real} = Real[],
        P_ramp_first::AbstractVector{<:Real} = Real[], P_ramp_last::AbstractVector{<:Real} = Real[];
        log_group::String = "")

    if length(partitions) <= 1
        return opf(Q, L_lines + L_gens, P_max, P_total, A_constraints, P_constraints,
            A_ramp, ΔP_ramp, P_ramp_first, P_ramp_last, log_group = log_group)
    end
    
    N = length(P_max)
    T = length(P_total)
    n_constraints = length(P_constraints)
    n_ramp = length(ΔP_ramp)

    # check dimensions of the input that needs to be partitioned
    @assert size(L_lines) == (N, T)
    @assert size(L_gens) == (N, T)
    @assert size(A_constraints) == (n_constraints, N)
    @assert (size(A_ramp) == (n_ramp, N)) || (n_ramp == 0)
    
    # check that the number of partitions matches the total number of steps
    @assert prod(partitions) == T
    
    n_partitions = partitions[1]
    partition_length = T ÷ n_partitions
    
    counter_width = length(string(n_partitions))    
    @info "Partitioning a dataset of $T time steps into $n_partitions chunks of $partition_length time steps" _group = log_group

    partitioned_P_total = reshape(P_total, (partition_length, n_partitions))
    aggregated_P_total = dropdims(sum(partitioned_P_total, dims=1), dims=1) / partition_length

    partitioned_L_lines = reshape(L_lines, (N, partition_length, n_partitions))
    aggregated_L_lines = dropdims(sum(partitioned_L_lines, dims=2), dims=2) / partition_length

    partitioned_L_gens = reshape(L_gens, (N, partition_length, n_partitions))
    
    aggregated_P = opf(Q, aggregated_L_lines, P_max, aggregated_P_total, A_constraints, P_constraints,
        A_ramp, ΔP_ramp, P_ramp_first, P_ramp_last,
        log_group = log_group * " $(lpad(0, counter_width))/$(n_partitions)")

    aggregated_P = min.(aggregated_P, P_max)
    partitioned_P_constraints = A_constraints * aggregated_P
    partitioned_P_ramp = n_ramp > 0 ? A_ramp * aggregated_P : Real[]

    result = Matrix{Float64}(undef, N, 0)
    timing = []
    for a=1:n_partitions

        if length(timing) > 0
            estimated_remaining_time = "Estimated remaining time:"
            s = round(Int, (n_partitions - a + 1) * sum(timing) / length(timing))
            if s >= 60
                m = s ÷ 60
                s = s % 60
                if m >= 60
                    h = m ÷ 60
                    m = m % 60
                    estimated_remaining_time = estimated_remaining_time * " $h h"
                end
                estimated_remaining_time = estimated_remaining_time * " $m min"
            end
            estimated_remaining_time = estimated_remaining_time * " $s s"
            @info estimated_remaining_time _group = log_group
        end
        
        if n_ramp == 0
            partitioned_P_ramp_previous = Real[]
            partitioned_P_ramp_next = Real[]
        else
            if a == 1
                partitioned_P_ramp_previous = length(P_ramp_first) > 0 ? P_ramp_first : partitioned_P_ramp[:, end]
            else
                partitioned_P_ramp_previous = partitioned_P_ramp[:, a - 1]
            end
            if a == n_partitions
                partitioned_P_ramp_next = length(P_ramp_last) > 0 ? P_ramp_last : partitioned_P_ramp[:, 1]
            else
                partitioned_P_ramp_next = partitioned_P_ramp[:, a + 1]
            end
        end
        
        partition_result = @timed partitioned_opf(partitions[2:end], Q, partitioned_L_lines[:,:,a], partitioned_L_gens[:,:,a],
            P_max, partitioned_P_total[:,a], A_constraints, partitioned_P_constraints[:,a],
            A_ramp, ΔP_ramp, partitioned_P_ramp_previous, partitioned_P_ramp_next,
            log_group = log_group * " $(lpad(a, counter_width))/$(n_partitions)")
        push!(timing, partition_result.time)
        result = hcat(result, partition_result.value)
    end

    return result
end

partitioned_opf (generic function with 5 methods)

## Example

Possible partitions:

In [18]:
52 * 168, 26 * 336, 13 * 672, 12 * 728, 8 * 1092, 4 * 2184

(8736, 8736, 8736, 8736, 8736, 8736)

In [19]:
P_total_reduced = dropdims(sum(reshape(P_total, (728, 12)), dims=1), dims=1) ./ 728
L_lines_reduced = dropdims(sum(reshape(L_lines, (1039, 728, 12)), dims=2), dims=2) ./ 728
L_gens_reduced = dropdims(sum(reshape(L_gens, (1039, 728, 12)), dims=2), dims=2) ./ 728;

In [20]:
P_full = opf(Q, L_lines_reduced + L_gens_reduced, P_max, P_total_reduced, A_constraints, P_constraints, A_ramp, ΔP_ramp);

[2024-05-28 15:50:22]  OPF with 12 time steps, 1039 generators, 1038 annual constraints, and 167 ramp constraints (cyclic)
[2024-05-28 15:50:22]   -> checking model
[2024-05-28 15:50:22]   -> defining variables
[2024-05-28 15:50:22]   -> defining constraints
[2024-05-28 15:50:23]   -> computing objective function
[2024-05-28 15:50:24]   -> optimizing
[2024-05-28 15:52:20]   -> exporting results


In [24]:
P_partitioned = partitioned_opf([3, 4], Q, L_lines_reduced, L_gens_reduced,
    P_max, P_total_reduced, A_constraints, P_constraints, A_ramp, ΔP_ramp);

[2024-05-28 16:00:57]  Partitioning a dataset of 12 time steps into 3 chunks of 4 time steps
[2024-05-28 16:00:57]  0/3 OPF with 3 time steps, 1039 generators, 1038 annual constraints, and 167 ramp constraints (cyclic)
[2024-05-28 16:00:57]       -> checking model
[2024-05-28 16:00:57]       -> defining variables
[2024-05-28 16:00:57]       -> defining constraints
[2024-05-28 16:00:57]       -> computing objective function
[2024-05-28 16:00:57]       -> optimizing
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 24.04 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 17343 rows, 21819 columns and 57645 nonzeros
Model fingerprint: 0x93c445d4
Model has 1620840 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [9e-02, 7e+01]
  QObjective range [3e-08, 2e+00]
  Bounds range     [1e-

In [20]:
P_doubly_partitioned = partitioned_opf([2, 2, 3], Q, L_lines_reduced, L_gens_reduced,
    P_max, P_total_reduced, A_constraints, P_constraints, A_ramp, ΔP_ramp);

[2024-05-28 15:56:22]  Partitioning a dataset of 12 time steps into 2 chunks of 6 time steps
[2024-05-28 15:56:22]  0/2 OPF with 2 time steps, 1039 generators, 1038 annual constraints, and 167 ramp constraints (cyclic)
[2024-05-28 15:56:22]       -> checking model
[2024-05-28 15:56:22]       -> defining variables
[2024-05-28 15:56:22]       -> defining constraints
[2024-05-28 15:56:23]       -> computing objective function
[2024-05-28 15:56:23]       -> optimizing
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 24.04 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1708 rows, 2078 columns and 5490 nonzeros
Model fingerprint: 0x34a6a8c5
Model has 1080560 quadratic objective terms
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [4e-02, 6e+01]
  QObjective range [3e-08, 2e+00]
  Bounds range     [1e-02,

## Comparison between partitions

In [23]:
using Plots

In [24]:
plot(P_full[312, :], label="Full", title="Bełchatów - Brown Lignite")
plot!(P_partitioned[312, :], label="Partitioned")
plot!(P_doubly_partitioned[312, :], label="Doubly partitioned")

LoadError: UndefVarError: `P_doubly_partitioned` not defined

## Turning off the noise

In [25]:
P_full_no_noise = opf(Q, L_lines_reduced, P_max, P_total_reduced, A_constraints, P_constraints, A_ramp, ΔP_ramp);

[2024-05-28 15:53:38]  OPF with 12 time steps, 1039 generators, 1038 annual constraints, and 167 ramp constraints (cyclic)
[2024-05-28 15:53:38]   -> checking model
[2024-05-28 15:53:38]   -> defining variables
[2024-05-28 15:53:38]   -> defining constraints
[2024-05-28 15:53:38]   -> computing objective function
[2024-05-28 15:53:40]   -> optimizing
[2024-05-28 15:55:05]   -> exporting results


In [26]:
P_partitioned_no_noise = partitioned_opf([3, 4], Q, L_lines_reduced, 0 * L_gens_reduced,
    P_max, P_total_reduced, A_constraints, P_constraints, A_ramp, ΔP_ramp);

[2024-05-28 15:55:05]  Partitioning a dataset of 12 time steps into 3 chunks of 4 time steps
[2024-05-28 15:55:05]  0/3 OPF with 3 time steps, 1039 generators, 1038 annual constraints, and 167 ramp constraints (cyclic)
[2024-05-28 15:55:05]       -> checking model
[2024-05-28 15:55:05]       -> defining variables
[2024-05-28 15:55:05]       -> defining constraints
[2024-05-28 15:55:05]       -> computing objective function
[2024-05-28 15:55:05]       -> optimizing
[2024-05-28 15:55:11]       -> exporting results
[2024-05-28 15:55:11]  1/3 OPF with 4 time steps, 1039 generators, 1038 annual constraints, and 167 ramp constraints (fixed boundaries)
[2024-05-28 15:55:11]       -> checking model
[2024-05-28 15:55:11]       -> defining variables
[2024-05-28 15:55:11]       -> defining constraints
[2024-05-28 15:55:11]       -> computing objective function
[2024-05-28 15:55:11]       -> optimizing
[2024-05-28 15:55:26]       -> exporting results
[2024-05-28 15:55:26]  Estimated remaining time

In [ ]:
P_doubly_partitioned_no_noise = partitioned_opf([2, 2, 3], Q, L_lines_reduced, 0 * L_gens_reduced,
    P_max, P_total_reduced, A_constraints, P_constraints, A_ramp, ΔP_ramp);

[2024-05-28 15:55:49]  Partitioning a dataset of 12 time steps into 2 chunks of 6 time steps
[2024-05-28 15:55:49]  0/2 OPF with 2 time steps, 1039 generators, 1038 annual constraints, and 167 ramp constraints (cyclic)
[2024-05-28 15:55:49]       -> checking model
[2024-05-28 15:55:49]       -> defining variables
[2024-05-28 15:55:49]       -> defining constraints
[2024-05-28 15:55:49]       -> computing objective function
[2024-05-28 15:55:49]       -> optimizing
[2024-05-28 15:55:52]       -> exporting results
[2024-05-28 15:55:52]  1/2 Partitioning a dataset of 6 time steps into 2 chunks of 3 time steps
[2024-05-28 15:55:52]  1/2 0/2 OPF with 2 time steps, 1039 generators, 1038 annual constraints, and 167 ramp constraints (fixed boundaries)
[2024-05-28 15:55:52]           -> checking model
[2024-05-28 15:55:52]           -> defining variables
[2024-05-28 15:55:52]           -> defining constraints
[2024-05-28 15:55:52]           -> computing objective function
[2024-05-28 15:55:53] 

In [ ]:
plot(P_full_no_noise[312, :], label="Full", title="Bełchatów - Brown Lignite")
plot!(P_partitioned_no_noise[312, :], label="Partitioned")
plot!(P_doubly_partitioned_no_noise[312, :], label="Doubly partitioned")

## Checks

Check that the total production for all generators is the same in all cases:

In [ ]:
[maximum(abs.(sum(P, dims=2) - sum(P_full, dims=2))) for P ∈ [P_partitioned, P_doubly_partitioned]]

In [ ]:
[maximum(abs.(sum(P, dims=2) - sum(P_full, dims=2)))
    for P ∈ [P_full_no_noise, P_partitioned_no_noise, P_doubly_partitioned_no_noise]]